In [1]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from huggingface_hub import login
from torch.nn.functional import softmax

In [2]:
login()

In [2]:
checkpoint="s-nlp/roberta_toxicity_classifier"
dataset="harsha2946/ParaDetox-Augmented"
ds=load_dataset(dataset)

In [3]:
model=AutoModelForSequenceClassification.from_pretrained(checkpoint, device_map="auto")
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

Some weights of the model checkpoint at s-nlp/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


In [5]:
def generate_toxicity_labels(ex):
    tokenized_tox_ex=tokenizer(ex["en_toxic_comment"], return_tensors="pt").to(model.device)
    logits=model(**tokenized_tox_ex).logits
    preds=softmax(logits, dim=-1)
    tox_toxicity=preds[0][1]
    tokenized_neu_ex=tokenizer(ex["en_neutral_comment"], return_tensors="pt").to(model.device)
    logits=model(**tokenized_neu_ex).logits
    preds=softmax(logits, dim=-1)
    neu_toxicity=preds[0][1]
    return {
        "toxic_comment_toxicity": tox_toxicity,
        "neutral_comment_toxicity": neu_toxicity
    }
    

In [ ]:
ds=ds.map(generate_toxicity_labels, batched=False)

Map:   0%|          | 0/39488 [00:00<?, ? examples/s]

In [ ]:
ds.push_to_hub(dataset)